In [2]:
import os
import random
import shutil
import json
from PIL import Image
import cv2
from ultralytics import YOLO
import pandas as pd

WARNING  Python>=3.10 is required, but Python==3.9.20 is currently installed 


Cambio de nombre de las imágenes

In [4]:
# Número actual de imágenes
imagenes_actuales = 125

# Ruta de la carpeta original que contiene las imágenes
ruta_original = "imagenes"

# Nombre de la nueva carpeta donde se guardarán las imágenes renombradas
nueva_carpeta = "Dataset/"

# Asegurarse de que la nueva carpeta existe o se crea
if not os.path.exists(nueva_carpeta):
    os.makedirs(nueva_carpeta)

# Recorrer las imágenes en la carpeta original
for i, nombre_archivo in enumerate(os.listdir(ruta_original)):
    # Verificar si el archivo es una imagen (puedes añadir más extensiones si lo necesitas)
    if nombre_archivo.endswith(('.jpg')):
        # Definir el nuevo nombre para la imagen (ejemplo: imagen_1.jpg)
        nuevo_nombre = f"imagen_{i+1+imagenes_actuales}.jpg"

        # Ruta completa del archivo original
        ruta_original_completa = os.path.join(ruta_original, nombre_archivo)

        # Ruta completa del nuevo archivo
        ruta_nueva_completa = os.path.join(nueva_carpeta, nuevo_nombre)

        # Copiar y renombrar la imagen a la nueva carpeta
        shutil.copy(ruta_original_completa, ruta_nueva_completa)

print("Proceso completado.")


Proceso completado.


Limpieza de los directorios train, val y test para añadir imagenes nuevamente

In [10]:
# Función para limpiar los directorios de imágenes y etiquetas
def limpiar_directorios(carpeta_destino):
    # Subcarpetas para los conjuntos de datos
    subcarpetas = ['train', 'test', 'val']
    
    # Tipos de archivo que queremos eliminar
    extensiones = ['.jpg', '.png', '.txt']  # Añade aquí otras extensiones de imagen si es necesario
    
    for subcarpeta in subcarpetas:
        carpeta_imagenes = os.path.join(carpeta_destino, subcarpeta, "images")
        carpeta_labels = os.path.join(carpeta_destino, subcarpeta, "labels")
        
        # Eliminar archivos en la carpeta de imágenes
        if os.path.exists(carpeta_imagenes):
            for archivo in os.listdir(carpeta_imagenes):
                if any(archivo.endswith(ext) for ext in extensiones):
                    os.remove(os.path.join(carpeta_imagenes, archivo))
        
        # Eliminar archivos en la carpeta de etiquetas
        if os.path.exists(carpeta_labels):
            for archivo in os.listdir(carpeta_labels):
                if archivo.endswith('.txt'):
                    os.remove(os.path.join(carpeta_labels, archivo))
    
    print("Los directorios de train, test y val han sido limpiados.")

# Ejemplo de uso
carpeta_destino = "VC_P4-NUM_PLATES"  # Ruta a la carpeta principal
limpiar_directorios(carpeta_destino)

Los directorios de train, test y val han sido limpiados.


Distribución de las imágenes entre train (70%), test (20%) y val (10%).

In [7]:
# Definir las clases y el ID de la clase "matricula"
clases = ["matricula"]

# Función para convertir al formato YOLO y escribir en el archivo .txt
def convertir_a_yolo(class_id, x_min, y_min, x_max, y_max, imagen_ancho, imagen_alto, archivo_txt):
    # Calcular coordenadas y tamaño del bounding box normalizado
    x_center = ((x_min + x_max) / 2) / imagen_ancho
    y_center = ((y_min + y_max) / 2) / imagen_alto
    width = (x_max - x_min) / imagen_ancho
    height = (y_max - y_min) / imagen_alto

    # Escribir en el archivo .txt en formato YOLO
    archivo_txt.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

# Ruta de la carpeta original que contiene las imágenes y JSON
carpeta_original = "Dataset"

# Carpeta de destino principal
carpeta_destino = "VC_P4-NUM_PLATES"

# Subcarpetas dentro de cada subconjunto
subcarpetas = ['train', 'test', 'val']
carpeta_images = {subcarpeta: os.path.join(carpeta_destino, subcarpeta, "images") for subcarpeta in subcarpetas}
carpeta_labels = {subcarpeta: os.path.join(carpeta_destino, subcarpeta, "labels") for subcarpeta in subcarpetas}

# Asegurarse de que las carpetas de destino existen o se crean
for subcarpeta in subcarpetas:
    os.makedirs(carpeta_images[subcarpeta], exist_ok=True)
    os.makedirs(carpeta_labels[subcarpeta], exist_ok=True)

# Obtener todas las imágenes de la carpeta original
imagenes = [f for f in os.listdir(carpeta_original) if f.endswith('.jpg')]

# Mezclar las imágenes de forma aleatoria
random.shuffle(imagenes)

# Calcular cuántas imágenes irán en cada subconjunto
total_imagenes = len(imagenes)
train_size = int(0.7 * total_imagenes)
test_size = int(0.2 * total_imagenes)
val_size = total_imagenes - train_size - test_size  # Lo que queda es para val

# Dividir las imágenes en tres subconjuntos
train_imagenes = imagenes[:train_size]
test_imagenes = imagenes[train_size:train_size + test_size]
val_imagenes = imagenes[train_size + test_size:]

# Función para copiar imágenes a las carpetas 'images' y crear archivos .txt con anotaciones en 'labels'
def copiar_imagenes_con_anotaciones(imagenes, carpeta_imagenes, carpeta_labels):
    for imagen in imagenes:
        # Copiar la imagen a la subcarpeta "images"
        ruta_origen_imagen = os.path.join(carpeta_original, imagen)
        ruta_destino_imagen = os.path.join(carpeta_imagenes, imagen)
        shutil.copy(ruta_origen_imagen, ruta_destino_imagen)

        # Generar la ruta del archivo JSON correspondiente
        nombre_sin_extension = os.path.splitext(imagen)[0]  # Nombre sin extensión
        archivo_json = f"{nombre_sin_extension}.json"
        ruta_origen_json = os.path.join(carpeta_original, archivo_json)

        # Crear el archivo .txt en la carpeta "labels" y convertir el JSON a formato YOLO, si existe el JSON
        ruta_txt = os.path.join(carpeta_labels, f"{nombre_sin_extension}.txt")
        if os.path.exists(ruta_origen_json):
            with open(ruta_txt, 'w') as archivo_txt, open(ruta_origen_json, 'r') as json_file:
                datos = json.load(json_file)

                # Abrir la imagen para obtener sus dimensiones
                with Image.open(ruta_destino_imagen) as img:
                    imagen_ancho, imagen_alto = img.size

                # Recorrer los objetos anotados en el JSON y escribir en formato YOLO
                for forma in datos['shapes']:
                    if forma['label'] == 'matricula' and forma['shape_type'] == 'rectangle':
                        puntos = forma['points']
                        x_min = min(p[0] for p in puntos)
                        y_min = min(p[1] for p in puntos)
                        x_max = max(p[0] for p in puntos)
                        y_max = max(p[1] for p in puntos)

                        # Llamar a la función que convierte al formato YOLO y escribe en el archivo .txt
                        class_id = clases.index('matricula')
                        convertir_a_yolo(class_id, x_min, y_min, x_max, y_max, imagen_ancho, imagen_alto, archivo_txt)

# Copiar las imágenes y crear los archivos .txt en las carpetas correspondientes
copiar_imagenes_con_anotaciones(train_imagenes, carpeta_images['train'], carpeta_labels['train'])
copiar_imagenes_con_anotaciones(test_imagenes, carpeta_images['test'], carpeta_labels['test'])
copiar_imagenes_con_anotaciones(val_imagenes, carpeta_images['val'], carpeta_labels['val'])

print("Las imágenes y archivos .txt de anotaciones han sido divididos en train, test y val.")

Las imágenes y archivos .txt de anotaciones han sido divididos en train, test y val.


Pruebas

In [8]:
# Carga del modelo entrenado
model = YOLO('runs/detect/train2/weights/best.pt')

# Ruta del video de entrada
filename = "C0142.MP4"

# Abrir el archivo de video
cap = cv2.VideoCapture(filename)

# Obtener información sobre el video
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Crear la ruta de salida para el video procesado
output_filename = 'video_con_detecciones_2.mp4'
output_path = os.path.join(os.getcwd(), output_filename)

# Crear un VideoWriter para guardar el video con detecciones
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

# Procesar el video frame a frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # Si no hay más frames, salir del bucle

    # Realizar detecciones en el frame actual
    results = model(frame)

    # Obtener el frame anotado con las detecciones
    annotated_frame = results[0].plot()

    # Guardar el frame anotado en el archivo de video de salida
    out.write(annotated_frame)

# Liberar los recursos
cap.release()
out.release()

print(f"Video guardado en {output_path}")


0: 384x640 (no detections), 119.3ms
Speed: 11.0ms preprocess, 119.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 199.8ms
Speed: 5.0ms preprocess, 199.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 145.2ms
Speed: 6.0ms preprocess, 145.2ms inference, 3.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 156.6ms
Speed: 5.6ms preprocess, 156.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 matricula, 113.0ms
Speed: 10.0ms preprocess, 113.0ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 matricula, 109.1ms
Speed: 4.0ms preprocess, 109.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 115.3ms
Speed: 6.0ms preprocess, 115.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 98.7ms
Speed: 5.0ms preprocess, 

Entrgable

In [ ]:
import cv2
import csv
import pandas as pd
from yolov5 import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import easyocr  # Biblioteca OCR para reconocimiento de texto en matrículas

# Inicializa el modelo de detección de objetos, OCR y seguimiento
yolo_model = YOLO("yolov5s.pt")
tracker = DeepSort(max_age=30, n_init=3)
ocr_reader = easyocr.Reader(['en'])  # Especifica el idioma que se va a usar para el OCR

# Inicializa el video de entrada y salida
video_input = cv2.VideoCapture("input_video.mp4")
video_output = cv2.VideoWriter("output_video.mp4", cv2.VideoWriter_fourcc(*'mp4v'), 30, (int(video_input.get(3)), int(video_input.get(4))))

# CSV donde se almacenará la información
csv_data = []

frame_number = 0

while video_input.isOpened():
    ret, frame = video_input.read()
    if not ret:
        break

    # Detecta objetos en el fotograma
    detections = yolo_model(frame)
    
    # Almacena las detecciones para DeepSort
    bboxes = []
    for det in detections:
        if det['class'] in ['person', 'car']:  # Filtra las clases de interés
            bboxes.append([det['x1'], det['y1'], det['x2'], det['y2'], det['confidence'], det['class']])

    # Aplica el seguimiento
    tracked_objects = tracker.update_tracks(bboxes, frame=frame)

    # Dibuja las detecciones y seguimiento en el frame
    for obj in tracked_objects:
        if not obj.is_confirmed():
            continue
        track_id = obj.track_id
        bbox = obj.to_tlbr()  # Bounding box
        class_name = obj.get_class()  # Clases

        # Si el objeto es un vehículo, intenta detectar la matrícula
        matricula_text = ""
        matricula_confianza = 0
        matricula_bbox = [None, None, None, None]

        if class_name == 'car':
            # Recortar la región de la matrícula y aplicar OCR
            vehicle_crop = frame[int(bbox[1]):int(bbox[3]), int(bbox[0]):int(bbox[2])]
            ocr_results = ocr_reader.readtext(vehicle_crop, detail=1, min_size=30)  # Cambia `min_size` según el tamaño de la matrícula

            # Procesar resultados OCR
            for (bbox_ocr, text, conf) in ocr_results:
                if conf > matricula_confianza:  # Seleccionar el resultado OCR con mayor confianza
                    matricula_text = text
                    matricula_confianza = conf
                    matricula_bbox = [bbox_ocr[0][0] + int(bbox[0]), bbox_ocr[0][1] + int(bbox[1]), 
                                      bbox_ocr[2][0] + int(bbox[0]), bbox_ocr[2][1] + int(bbox[1])]
        
        # Agrega la entrada al CSV
        csv_data.append([frame_number, class_name, obj.confidence, track_id, *bbox, matricula_text, matricula_confianza, *matricula_bbox])

        # Dibuja la caja y el ID
        cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (255, 0, 0), 2)
        cv2.putText(frame, f"ID: {track_id} {class_name}", (int(bbox[0]), int(bbox[1] - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        # Dibuja la matrícula detectada
        if matricula_text:
            cv2.putText(frame, matricula_text, (int(matricula_bbox[0]), int(matricula_bbox[1] - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
            cv2.rectangle(frame, (int(matricula_bbox[0]), int(matricula_bbox[1])), (int(matricula_bbox[2]), int(matricula_bbox[3])), (0, 255, 255), 2)

    # Guarda el frame anotado en el video de salida
    video_output.write(frame)
    frame_number += 1

# Guarda el CSV
df = pd.DataFrame(csv_data, columns=['fotograma', 'tipo_objeto', 'confianza', 'identificador_tracking', 'x1', 'y1', 'x2', 'y2', 'matrícula_en_su_caso', 'confianza_matricula', 'mx1', 'my1', 'mx2', 'my2'])
df.to_csv("output_data.csv", index=False)

# Libera los recursos
video_input.release()
video_output.release()